<a href="https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sudoku Reader

In this project a software is developed which recognizes a Sudoku puzzle and solves it automatically.

![Sudoku Puzzle](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/Aufgabe.jpg?raw=1)

First milestone: **Sudoku square is found**

1. Define use cases
> In which use cases is a Sudoku solver used?

2. Image Preprocessing
> Read in the image (Sudoku puzzle) with OpenCV

3. Find Sudoku Square
> Find the outer corners of the puzzle to recognize Sudoku puzzles.

4. Find vertical and horizontal lines

5. Merge vertical and horizontal lines
> The squares in the puzzle can be found with grid points

Second milestone: **Digits in Sudoku puzzles are recognized**

1. Recognize digital digits
> Maybe with template matching from OpenCV

2. Create a Convolutional neural network

3. Train CNN with the MNIST dataset
> The MNIST dataset is used to detect handwritten digits.

4. Evaluate CNN
> Check if predictions are correct

Third milestone: **Sudoku Solver completed**

1. Convert Sudoku to Matrix
> For solving the puzzle

2. Search and integrate a good existing solver 
> An algorithm that solves and fills the empty spaces in the puzzle

3. Transfer the solution to augmented reality